In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

/Users/andiklausli/opt/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/andiklausli/opt/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/andiklausli/opt/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/andiklausli/opt/miniconda3/lib/python3.7/s

In [2]:
df = pd.read_csv('data_spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Data Cleansing

In [3]:
df.info() # non NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [50]:
# df = pd.get_dummies(df, columns=['Category'])

KeyError: "None of [Index(['Category'], dtype='object')] are in the [columns]"

In [4]:
df.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# replace Category 
dict_trans = {'ham' : 0, 'spam': 1}
df = df.replace({"Category": dict_trans})

In [7]:
df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Data Preparation

In [8]:
# train-/test-split

labels = df['Category'].values
messages = df['Message'].values
messages_train, messages_test, y_train, y_test = train_test_split(messages, labels, test_size=0.3, random_state=42)

array([0, 0, 1, ..., 0, 0, 0])

In [9]:
# Vectorisation
vectorizer = CountVectorizer()
vectorizer.fit(messages_train)

X_train = vectorizer.transform(messages_train)
X_test  = vectorizer.transform(messages_test)

In [10]:
X_train

<3900x7262 sparse matrix of type '<class 'numpy.int64'>'
	with 52129 stored elements in Compressed Sparse Row format>

# Build First Model

In [12]:

input_dim = X_train.shape[1]  # Number of features

In [13]:
input_dim

7262

In [14]:

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [15]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                72630     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 72,641
Trainable params: 72,641
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

2021-09-09 11:55:34.478237: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-09 11:55:34.480021: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 16. Tune using inter_op_parallelism_threads for best performance.


Train on 3900 samples, validate on 1672 samples
Epoch 1/100
3900/3900 [==============================] - 2s 465us/step - loss: 0.2784 - accuracy: 0.9392 - val_loss: 0.1084 - val_accuracy: 0.9850
Epoch 2/100
3900/3900 [==============================] - 1s 327us/step - loss: 0.0666 - accuracy: 0.9882 - val_loss: 0.0621 - val_accuracy: 0.9892
Epoch 3/100
3900/3900 [==============================] - 1s 330us/step - loss: 0.0302 - accuracy: 0.9946 - val_loss: 0.0503 - val_accuracy: 0.9898
Epoch 4/100
3900/3900 [==============================] - 1s 329us/step - loss: 0.0158 - accuracy: 0.9982 - val_loss: 0.0463 - val_accuracy: 0.9892
Epoch 5/100
3900/3900 [==============================] - 1s 332us/step - loss: 0.0100 - accuracy: 0.9990 - val_loss: 0.0459 - val_accuracy: 0.9892
Epoch 6/100
3900/3900 [==============================] - 1s 331us/step - loss: 0.0069 - accuracy: 0.9992 - val_loss: 0.0480 - val_accuracy: 0.9898
Epoch 7/100
3900/3900 [==============================] - 1s 365us/step

3900/3900 [==============================] - 1s 344us/step - loss: 3.0438e-04 - accuracy: 1.0000 - val_loss: 0.1551 - val_accuracy: 0.9880
Epoch 56/100
3900/3900 [==============================] - 1s 338us/step - loss: 2.9506e-04 - accuracy: 1.0000 - val_loss: 0.1579 - val_accuracy: 0.9880
Epoch 57/100
3900/3900 [==============================] - 1s 360us/step - loss: 2.8752e-04 - accuracy: 1.0000 - val_loss: 0.1608 - val_accuracy: 0.9880
Epoch 58/100
3900/3900 [==============================] - 1s 334us/step - loss: 2.8068e-04 - accuracy: 1.0000 - val_loss: 0.1645 - val_accuracy: 0.9880
Epoch 59/100
3900/3900 [==============================] - 1s 355us/step - loss: 2.7332e-04 - accuracy: 1.0000 - val_loss: 0.1655 - val_accuracy: 0.9880
Epoch 60/100
3900/3900 [==============================] - 1s 362us/step - loss: 2.6564e-04 - accuracy: 1.0000 - val_loss: 0.1668 - val_accuracy: 0.9880
Epoch 61/100
3900/3900 [==============================] - 2s 388us/step - loss: 2.5833e-04 - accuracy

In [17]:
# model performance
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 1.0000
Testing Accuracy:  0.9862
